In [ ]:
# Import tensorflow and pyplot
import tensorflow as tf
import matplotlib.pyplot as plt

1. tf.Graph represents a collection of tf.Operations
2. You can create operations by writing out equations.
3. By default, there is a graph: tf.get_default_graph() and any new operations are added to this graph.
4. The result of a tf.Operation is a tf.Tensor, which holds the values.

In [ ]:
# First a tf.Tensor
n_values = 32
x = tf.linspace(-3.0, 3.0, n_values)

In [ ]:
# Construct a tf.Session to execute the graph.
sess = tf.Session()
result = sess.run(x)

In [ ]:
# Alternatively pass a session to the eval fn:
x.eval(session=sess)
# x.eval() does not work, as it requires a session!

In [ ]:
# We can setup an interactive session if we don't
# want to keep passing the session around:
sess.close()
sess = tf.InteractiveSession()

In [ ]:
# Now this will work!
x.eval()

In [ ]:
# Now a tf.Operation
# We'll use our values from [-3, 3] to create a Gaussian Distribution
sigma = 1.0
mean = 0.0
z = (tf.exp(tf.negative(tf.pow(x - mean, 2.0)/(2.0 * tf.pow(sigma, 2.0)))) *(1.0 / (sigma * tf.sqrt(2.0 * 3.1415))))

In [ ]:
# By default, new operations are added to the default Graph
assert z.graph is tf.get_default_graph()



In [ ]:
# Execute the graph and plot the result
plt.plot(x.eval(), z.eval())
plt.show()

In [ ]:
# We can find out the shape of a tensor like so:
print(z.get_shape())

In [ ]:
# Or in a more friendly format
print(z.get_shape().as_list())

In [ ]:
#Sometimes we may not know the shape of a tensor
# until it is computed in the graph.  In that case
# we should use the tf.shape fn, which will return a
# Tensor which can be eval'ed, rather than a discrete
# value of tf.Dimension
print(tf.shape(z).eval())

In [ ]:
# %% Let's multiply the two to get a 2d gaussian
z_2d = tf.matmul(tf.reshape(z, [n_values, 1]), tf.reshape(z, [1, n_values]))

In [ ]:
plt.imshow(z_2d.eval())
plt.show()

In [ ]:
x = tf.reshape(tf.sin(tf.linspace(-3.0, 3.0, n_values)), [n_values, 1])
y = tf.reshape(tf.ones_like(x), [1, n_values])
z = tf.multiply(tf.matmul(x, y), z_2d)
plt.imshow(z.eval())
plt.show()

In [ ]:
# We can also list all the operations of a graph:
ops = tf.get_default_graph().get_operations()
print([op.name for op in ops])
# run this yourself, some nasty values emerges here.

In [ ]:
# Lets try creating a generic function for computing the same thing:
def gabor(n_values=32, sigma=1.0, mean=0.0):
    x = tf.linspace(-3.0, 3.0, n_values)
    z = (tf.exp(tf.negative(tf.pow(x - mean, 2.0) /
                       (2.0 * tf.pow(sigma, 2.0)))) *
         (1.0 / (sigma * tf.sqrt(2.0 * 3.1415))))
    gauss_kernel = tf.matmul(
        tf.reshape(z, [n_values, 1]), tf.reshape(z, [1, n_values]))
    x = tf.reshape(tf.sin(tf.linspace(-3.0, 3.0, n_values)), [n_values, 1])
    y = tf.reshape(tf.ones_like(x), [1, n_values])
    gabor_kernel = tf.multiply(tf.matmul(x, y), gauss_kernel)
    return gabor_kernel

In [ ]:
# Lets's plot
plt.imshow(gabor().eval())
plt.show()

In [ ]:
# And another function which can convolve
def convolve(img, W):
    # The W matrix is only 2D
    # But conv2d will need a tensor which is 4d:
    # height x width x n_input x n_output
    if len(W.get_shape()) == 2:
        dims = W.get_shape().as_list() + [1, 1]
        W = tf.reshape(W, dims)

    if len(img.get_shape()) == 2:
        # num x height x width x channels
        dims = [1] + img.get_shape().as_list() + [1]
        img = tf.reshape(img, dims)
    elif len(img.get_shape()) == 3:
        dims = [1] + img.get_shape().as_list()
        img = tf.reshape(img, dims)
        # if the image is 3 channels, then our convolution
        # kernel needs to be repeated for each input channel
        W = tf.concat(2, [W, W, W])

    # Stride is how many values to skip for the dimensions of
    # num, height, width, channels
    convolved = tf.nn.conv2d(img, W,
                             strides=[1, 1, 1, 1], padding='SAME')
    return convolved


In [ ]:
# Load up an image:
from skimage import data
img = data.astronaut()
plt.imshow(img)
plt.show()
print(img.shape)